In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import pandas as pd

In [ ]:
# LAM = 0.02
with open('end_price_tol.pickle', 'rb') as fp:
    Two_price = pickle.load(fp)

with open('Q_hist_tol.pickle', 'rb') as fp:
    Two_heat = pickle.load(fp)

In [ ]:
cycles = np.zeros(10)
for k in range(len(Two_price)):
    cycles[k] = len(np.unique(Two_price[k]))
    print(len(np.unique(Two_price[k])))

In [ ]:
Two_ratio.mean(), Two_ratio.std()

In [4]:
n_instance = 5
n_agents = 3
actions_space = np.arange(1.2, 2.1, 0.04)
quality = np.ones(n_agents) * 2
margin_cost = np.ones(n_agents)
horizon = 1 / 4
a0 = 0
n_actions = actions_space.size
state_ravel = (n_actions,) * n_agents


def replay_classic_reward(price):
    # Compute profits for all agents
#     price = actions_space[action]
    demand = np.exp((quality - price) / horizon)
    demand = demand / (np.sum(demand) + np.exp(a0 / horizon))
    reward = np.multiply(price - margin_cost, demand)
    return reward

In [3]:
with open('ThreeAgents_Q.pickle', 'rb') as fp:
    Three_Q = pickle.load(fp)

with open('ThreeAgents_price.pickle', 'rb') as fp:
    Three_price = pickle.load(fp)

with open('ThreeAgents_LAM002_Q.pickle', 'rb') as fp:
    Three002_Q = pickle.load(fp)

with open('ThreeAgents_LAM002_price.pickle', 'rb') as fp:
    Three002_price = pickle.load(fp)

In [ ]:
np.unravel_index(Three_price[0],state_ravel)

In [ ]:
Three_price[0].shape

In [5]:
monopoly = replay_classic_reward(np.array([2.0, 2.0, 2.0]))[0]
nash = replay_classic_reward(np.array([1.37, 1.37, 1.37]))[0]

In [6]:
monopoly, nash

(0.25, 0.12011194666838046)

In [7]:
# Ratio matrix calculator
N = 200000

# Three agents 
# ss = np.random.SeedSequence(12345)
# # Last a few rngs for sampling memories
# child_seeds = ss.spawn(n_agents)
# rng = [np.random.default_rng(s) for s in child_seeds]

sim_ratio = np.zeros(n_instance)

converged = np.zeros(n_agents)
for ins in range(n_instance):
    sim_Q = Three002_Q[ins]
    state = Three002_price[ins][-1]
    action = np.zeros(n_agents, dtype=int)
    reward = np.zeros(n_agents)
    for k in range(N):
#         if k > N - 30:
#             print(state)
        # For each agent, select and perform an action
        for i in range(n_agents):
            action[i] = sim_Q[i, state].argmax()
        if k > N - 10001:
            reward += replay_classic_reward(actions_space[action])
            converged += actions_space[action]
        # Move to the next state
        state = np.ravel_multi_index(action, state_ravel)
        avg = np.sum(reward)/10001/n_agents
        sim_ratio[ins] = (avg - nash)/(monopoly - nash)
    print('Instance', ins, avg, 'ratio', sim_ratio[ins])
    print('Reward', reward)

Instance 0 0.13306518118787689 ratio 0.09972614253002382
Reward [1207.01098003 1298.09850948 1487.24514167]
Instance 1 0.13601727497360994 ratio 0.12245412797604484
Reward [ 878.09402976 1658.98338244 1543.84888883]
Instance 2 0.11713813971550284 ratio -0.022895153762025666
Reward [1204.3249404  1330.32791083  979.84275465]
Instance 3 0.13501089990569567 ratio 0.11470610926223082
Reward [1278.00108373 1597.86132937 1174.86961677]
Instance 4 0.11101951077576477 ratio -0.07000209533821886
Reward [1153.56314047 1028.28082106 1149.07442028]


In [8]:
sim_ratio

array([ 0.09972614,  0.12245413, -0.02289515,  0.11470611, -0.0700021 ])

In [9]:
# ThreeAgents Ratio
Three002_ratio = np.array([0.09972614, 0.12245413, -0.02289515, 0.11470611, -0.0700021])
Three01_ratio = np.array([0.45236368, 0.68260603, 0.54546982, 0.51089425, 0.54174672])
# LAM = 1/MEM_SIZE/0.05 Simulated Ratio
Two002_ratio = np.array([0.75023822, 0.93250101, 0.93591032, 0.41974075, 0.90971576,
                         0.92656322, 0.73407456, 0.84459248, 0.98900363, 0.84945931])

In [10]:
Three002_ratio.mean(), Three002_ratio.std()

(0.048797826, 0.07951874170406587)

In [11]:
Two002_ratio.mean(), Two002_ratio.std()

(0.8291799259999999, 0.15720872397514762)

In [12]:
Three01_ratio.mean(), Three01_ratio.std()

(0.5466160999999999, 0.07573355283374285)

In [ ]:
df1 = pd.DataFrame({'Ratio':Two_ratio, 'Agents':np.ones(10)*2})
df2 = pd.DataFrame({'Ratio':Three_ratio, 'Agents':np.ones(5)*3})
frames = [df1, df2]
df = pd.concat(frames)
sns.set_theme(style="whitegrid")
ax = sns.boxplot(x="Agents", y="Ratio", data=df)
ax.set_xlabel("Number of Agents") # , fontsize=14)
ax.set_ylabel("Profit Ratio")
# plt.savefig('three_agents.eps', format='eps', dpi=500, bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:
cycles = np.zeros(5)
for k in range(len(Three_price)):
    cycles[k] = len(np.unique(Three_price[k]))
    print(len(np.unique(Three_price[k])))